In [12]:
import numpy as np
import gym
from numpy.random import choice
import random
from phi.api import *
import tensorflow as tf
from tfinterface.reinforcement import DeepActorCritic, ExpandedStateEnv
from tfinterface.interfaces import EnvironmentInterface
from tfinterface.model_base import ModelBase
from tensorflow.python import debug as tf_debug
import os
from scipy.interpolate import interp1d
import numbers


def get_run():
    try:
        with open("run.txt") as f:
            run = int(f.read().split("/n")[0])
    except:
        run = -1
    
    with open("run.txt", 'w+') as f:
        run += 1
        
        f.seek(0)
        f.write(str(run))
        f.truncate()
        
    return run

In [30]:
run = get_run()
env = ExpandedStateEnv("CartPole-v1", 3)
print(env.action_space)
n_actions = env.action_space.n
n_states = env.observation_space.shape[0] * 3
model_path = os.getcwd() + "/actor-critic.model"
logs_path = "logs/run{}".format(run)

print("Run: {}".format(run))

[2017-03-04 08:38:02,318] Making new env: CartPole-v1


True
Discrete(2)
Run: 202


In [31]:
model = DeepActorCritic(
    n_actions, n_states, y=0.99999, 
    buffer_length=1000000, pi=0.02,
    model_path = model_path,
    logs_path = logs_path
)

In [32]:
k = 40000.
model.fit(
    env, keep_prob=0.5,print_step=10, 
    episodes=int(1e5), max_episode_length=200e3, batch_size=32,
    learning_rate = 0.01 # lambda t: 0.05 * k / (k + t)
)

[MAX] Episode: 0, Length: 24, Reward: 24.0, buffer_len: 24
[MAX] Episode: 2, Length: 33, Reward: 33.0, buffer_len: 80
[MAX] Episode: 3, Length: 55, Reward: 55.0, buffer_len: 135
[MAX] Episode: 6, Length: 55, Reward: 55.0, buffer_len: 265
[NOR] Episode: 10, Length: 31, Avg Reward: 36.3, Learning Rate: 0.01, buffer_len: 363
Loss: 0.467469424009
[MAX] Episode: 16, Length: 58, Reward: 58.0, buffer_len: 578
[MAX] Episode: 20, Length: 59, Reward: 59.0, buffer_len: 759
[NOR] Episode: 20, Length: 59, Avg Reward: 39.6, Learning Rate: 0.01, buffer_len: 759
Loss: 0.493125587702
[MAX] Episode: 26, Length: 98, Reward: 98.0, buffer_len: 1029
[NOR] Episode: 30, Length: 45, Avg Reward: 38.9, Learning Rate: 0.01, buffer_len: 1148
Loss: 0.613241314888
[NOR] Episode: 40, Length: 26, Avg Reward: 30.1, Learning Rate: 0.01, buffer_len: 1449
Loss: 0.518278598785
[NOR] Episode: 50, Length: 16, Avg Reward: 35.3, Learning Rate: 0.01, buffer_len: 1802
Loss: 0.616812705994
[NOR] Episode: 60, Length: 37, Avg Rewar

KeyboardInterrupt: 

In [35]:
import time

model_run = DeepActorCritic(
    n_actions, n_states,
    model_path = model_path + ".max",
    flush_secs = 3.0,
    restore = True
)
env = ExpandedStateEnv("CartPole-v1", 3)
s = env.reset()
done = False
total = 0
while not done:
    total += 1
    a = model_run.choose_action(s, 1.0, e=0.2)
    s, r, done, info = env.step(a)
    env.render()
    time.sleep(0.01)
    
print total

[2017-03-04 08:53:24,278] Making new env: CartPole-v1


True


ArgumentError: argument 2: <type 'exceptions.TypeError'>: wrong type